## 주제 
- 위성 소형화, 저궤도 위성, 광학위성

In [27]:
"""
담당자별 에이전트:
- 현준: startup_search_agent (스타트업 탐색)
- 승연: tech_summary_agent (기술 요약)
- 민주: market_eval_agent (시장성 평가)
- 경남: investor_insight_agent (투자자 인사이트)

+ investment_decision_agent (투자 판단)
"""

'\n담당자별 에이전트:\n- 현준: startup_search_agent (스타트업 탐색)\n- 승연: tech_summary_agent (기술 요약)\n- 민주: market_eval_agent (시장성 평가)\n- 경남: investor_insight_agent (투자자 인사이트)\n\n+ investment_decision_agent (투자 판단)\n'

In [28]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
from typing import Annotated, TypedDict, Sequence, Literal
from pydantic import BaseModel

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_react_agent

from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [ ]:
class AgentState(TypedDict):
    startup_name: str
    startup_info: dict
    tech_summary: str
    market_analysis: str
    investment_decision: str
    report: str
    iteration_count: int


## 스타트업 탐색 에이전트 (담당자: 현준)
1. 웹서치로 스타트업 기본 정보 수집


In [ ]:
def startup_search_agent(state: AgentState) -> AgentState:
    """
    스타트업 정보를 수집(또는 하드코딩)하여 state에 저장하는 함수
    """
    # 스타트업 이름
    state["startup_name"] = "Starlink"

    # 상세 정보 (dict 형태)
    state["startup_info"] = {
        "website": "https://www.starlink.com",
        "tech_focus": "저궤도 위성",
        "description": (
            "Starlink는 LEO 위성 시장에서 가장 큰 플레이어로, "
            "7,000개 이상의 활성 위성을 운영하고 있습니다. "
            "이들은 전 세계에 고속 인터넷 서비스를 제공하는 데 중점을 두고 있습니다."
        ),
        "founded_year": 2018,
        "location": "미국",
        "funding": "비공식적이며, 수십억 달러의 투자 유치",
    }

    return state

## 기술 요약 에이전트 (담당자: 승연)

In [ ]:
def tech_summary_agent(state: AgentState) -> AgentState:
    """승연 담당: 기술력 핵심 요약"""

    state["tech_summary"] = f"{company_name}의 기술 분석 결과 (승연 담당)"

    return state

## 시장성 평가 에이전트 (민주)

In [ ]:
from __future__ import annotations
from typing import Dict, Any, List
import os, json, re
from datetime import datetime
from pathlib import Path

from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


# ==========================================================
# 점수 가중치 정의 (총 100점 기준)
# ----------------------------------------------------------
# market_size: 시장규모(크기·확장성)
# growth: 성장률(산업 성장 속도)
# demand_signals: 수요 신호(정부/민간 수요)
# entry_barriers: 진입장벽(낮을수록 좋음)
# policy_budget_tailwind: 정책/예산 순풍(정부 지원 강도)
# competition_intensity: 경쟁 강도(낮을수록 좋음)
# gtm_feasibility: 시장 진입(GTM) 용이성
# ==========================================================
WEIGHTS: Dict[str, int] = {
    "market_size": 20,
    "growth": 15,
    "demand_signals": 15,
    "entry_barriers": 10,
    "policy_budget_tailwind": 15,
    "competition_intensity": 10,
    "gtm_feasibility": 15,
}


# ==========================================================
# 점수 계산 함수
# ----------------------------------------------------------
# - 누락 시 기본 55점
# - 진입장벽(entry_barriers)·경쟁강도(competition_intensity)는
#   낮을수록 유리하므로 100 - 점수로 반전
# ==========================================================
def _total_score(scores: Dict[str, float]) -> float:
    adj = {k: float(scores.get(k, 55)) for k in WEIGHTS}
    adj["entry_barriers"] = 100 - adj["entry_barriers"]
    adj["competition_intensity"] = 100 - adj["competition_intensity"]
    return round(sum(adj[k] * WEIGHTS[k] / 100 for k in WEIGHTS), 1)


# ==========================================================
# 밴드(Band) 정의
# ----------------------------------------------------------
# - Invest (≥80): 투자 유망
# - Watch (60~79): 긍정적이나 추세 관찰 필요
# - Hold (<60): 리스크 높아 보류
# ==========================================================
def _band(total: float) -> str:
    if total >= 80:
        return "Invest"
    elif total >= 60:
        return "Watch"
    return "Hold"


# ==========================================================
# Retriever (RAG 검색기) 생성
# ----------------------------------------------------------
# - PDF: 2024년 우주산업실태조사 보고서, 우주청 2025년 예산 편성안
# - HuggingFace 임베딩 + FAISS 벡터 검색 기반
# ==========================================================
_RETRIEVER = None


def _ensure_retriever(k: int = 8):
    global _RETRIEVER
    if _RETRIEVER is not None:
        return _RETRIEVER

    pdf_paths = [
        "data/2024년 우주산업실태조사 보고서(최종본).pdf",
        "data/우주청_2025년_예산_편성안.pdf",
    ]

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs_all = []

    for p in pdf_paths:
        if not os.path.exists(p):
            raise FileNotFoundError(f"PDF 파일이 존재하지 않습니다: {p}")
        loader = PyPDFLoader(p)
        pages = loader.load()
        for d in pages:
            d.metadata["source"] = Path(p).name
        docs_all.extend(splitter.split_documents(pages))

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    vectordb = FAISS.from_documents(docs_all, embeddings)
    _RETRIEVER = vectordb.as_retriever(search_kwargs={"k": k})
    return _RETRIEVER


# ==========================================================
# 컨텍스트 수집 (RAG)
# ----------------------------------------------------------
# - LLM이 참고할 문맥을 PDF에서 자동으로 검색
# - 시장 규모, 예산, 수요, 경쟁 등 4개 축으로 질의 수행
# ==========================================================
def _gather_context(target: str, topk_each: int = 6) -> str:
    retriever = _ensure_retriever()
    queries = [
        f"한국 {target} 시장 규모 성장률 활동금액",
        f"{target} 정부 예산 신규 사업 우주청 2025 계획",
        f"{target} 수요 조달 활용서비스 재난 기상 농업",
        f"{target} 경쟁 진입장벽 국내 기업 동향",
    ]

    docs: List[Document] = []
    for q in queries:
        try:
            docs.extend(retriever.get_relevant_documents(q))
        except Exception:
            continue

    # 중복 제거 (파일명 + 본문 일부 해시)
    seen, uniq = set(), []
    for d in docs:
        src = d.metadata.get("source") or ""
        key = (src, hash(d.page_content[:200]))
        if key not in seen:
            uniq.append(d)
            seen.add(key)
    uniq = uniq[:topk_each]

    blocks = []
    for d in uniq:
        src = d.metadata.get("source") or ""
        text = d.page_content.strip().replace("\n", " ")
        blocks.append(f"[{src}] {text}")
    return "\n\n".join(blocks)


# ==========================================================
# JSON 파싱 유틸
# ----------------------------------------------------------
# - LLM의 출력에서 JSON 블록만 추출해 Python dict로 변환
# ==========================================================
def _extract_json(text: str) -> Dict[str, Any]:
    if not text:
        return {}
    s = text.strip()
    m = re.search(r"```(?:json)?\s*(.*?)\s*```", s, flags=re.DOTALL)
    if m:
        s = m.group(1).strip()
    b = re.search(r"\{.*\}", s, flags=re.DOTALL)
    if b:
        s = b.group(0)
    try:
        return json.loads(s)
    except Exception:
        return {}


# ==========================================================
# 시장성 평가 에이전트
# ----------------------------------------------------------
# - RAG 기반으로 시장성 점수카드를 생성
# - state["startup_name"]을 입력받아 평가 JSON 반환
# ==========================================================
def market_eval_agent(state: AgentState) -> AgentState:
    """PDF 기반 RAG로 시장성 평가 수행"""

    target = state.get("startup_name") or "저궤도 위성"
    context = _gather_context(target)

    SYSTEM_PROMPT = (
        "당신은 한국 우주(위성) 산업의 시장평가 전문 애널리스트입니다. "
        "정부·산업 리포트를 근거로 ‘시장성 평가 점수카드(JSON)’를 작성하십시오.\n\n"
        "평가 원칙:\n"
        "1. 모든 평가는 객관적 근거 기반으로 작성\n"
        "2. 수치·경향은 명확히 기술하고 출처(파일명) 명시\n"
        "3. 항목별 점수는 0~100 범위로 부여\n"
        "4. 최종 출력은 완전한 JSON이어야 함"
    )

    USER_PROMPT = f"""
[분석 대상]
세그먼트: {target}

[컨텍스트]
{context}

[출력 형식(JSON 예시)]
{{
  "target": "{target}",
  "geo": "KR",
  "scores": {{
    "market_size": 0-100,
    "growth": 0-100,
    "demand_signals": 0-100,
    "entry_barriers": 0-100,
    "policy_budget_tailwind": 0-100,
    "competition_intensity": 0-100,
    "gtm_feasibility": 0-100
  }},
  "total": 0-100,
  "band": "Invest|Watch|Hold",
  "rationale": "핵심 요약",
  "key_evidence": [{{"claim": "string", "source": "filename"}}],
  "risks": ["string"],
  "assumptions": ["string"],
  "data_freshness": "as of YYYY-MM-DD"
}}
※ 반드시 유효한 JSON만 반환하십시오.
"""

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    result = llm.invoke(
        [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ]
    )

    data = _extract_json(getattr(result, "content", ""))

    # JSON 파싱 실패 시 기본값 적용
    if not isinstance(data, dict) or "scores" not in data:
        data = {
            "target": target,
            "geo": "KR",
            "scores": {k: 55 for k in WEIGHTS},
            "rationale": "기본값 적용(LLM 파싱 실패)",
            "key_evidence": [],
            "risks": [],
            "assumptions": [],
        }

    # 점수 보정 및 총점 계산
    for k in WEIGHTS:
        data["scores"][k] = float(data["scores"].get(k, 55))

    total = _total_score(data["scores"])
    data["total"] = total
    data["band"] = _band(total)
    data["data_freshness"] = f"as of {datetime.now().date()}"

    # 결과 저장
    state["market_analysis"] = json.dumps(data, ensure_ascii=False, indent=2)
    print(f"[시장성 평가 완료] {target} → 총점 {total}점 / 밴드 {data['band']}")
    return state

In [ ]:
state = {
    "startup_name": "저궤도 위성",
    "startup_info": {},
    "tech_summary": "",
    "market_analysis": "",
    "investment_decision": "",
    "report": "",
    "iteration_count": 0
}

state = market_eval_agent(state)
print(state["market_analysis"])

## 투자 판단 에이전트


In [ ]:
def investment_decision_agent(state: AgentState) -> AgentState:
    """경남 담당: 투자자 인사이트 분석 + Scorecard Method 기반 투자 판단"""

    company_name = state["startup_name"]
    tech_summary = state["tech_summary"]
    market_analysis = state["market_analysis"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    combined_prompt = f"""
    당신은 세계적인 벤처캐피탈리스트입니다.
    
    회사: {company_name}
    기술 분석: {tech_summary}
    시장 분석: {market_analysis}
    
    ## Part 1: 투자자 인사이트
    다음 관점에서 분석하세요:
    1. 비전과 시장 타이밍
    2. 팀의 실행력 가능성
    3. 스케일업 잠재력
    4. 경쟁 우위 지속가능성
    
    ## Part 2: Scorecard Method 평가
    각 항목을 50-150% 척도로 평가:
    - 제품/기술: 15-20%
    - 시장 기회: 20-25%
    - 팀: 25-30%
    - 경쟁환경: 10-15%
    
    형식:
    [투자자 인사이트]
    - 비전과 타이밍: ...
    - 실행력: ...
    - 스케일업: ...
    - 경쟁우위: ...
    
    [Scorecard 평가]
    기술: X%
    시장: Y%
    팀: Z%
    경쟁: W%
    
    [최종 결론]
    invest 또는 hold
    """

    result = llm.invoke(combined_prompt)
    evaluation_text = result.content

    if "invest" in evaluation_text.lower() and "hold" not in evaluation_text.lower():
        state["investment_decision"] = "invest"
    else:
        state["investment_decision"] = "hold"

    state["iteration_count"] = state.get("iteration_count", 0) + 1

    return state


# %%
def should_continue(state: AgentState) -> str:
    """조건 분기"""
    if state["investment_decision"] == "invest":
        return "report_writer"
    elif state["iteration_count"] >= 3:
        return "report_writer"
    return "startup_search"

## 보고서 생성 에이전트

In [ ]:
def report_writer_agent(state: AgentState) -> AgentState:
    """
    결과 요약 보고서 생성
    """
    print("=== 보고서 생성 에이전트 실행 ===")

    company_name = state["startup_name"]
    company_info = state["startup_info"]
    tech_summary = state["tech_summary"]
    market_analysis = state["market_analysis"]
    investment_decision = state["investment_decision"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    report_prompt = f"""
    당신은 전문 벤처 투자 분석가입니다.
    아래 분석 결과를 바탕으로 다음 형식에 맞춰 투자 평가 보고서를 작성하세요.
    
    === 분석 정보 ===
    
    회사명: {company_name}
    회사 정보: {company_info}
    
    기술력 분석:
    {tech_summary}
    
    시장성 분석:
    {market_analysis}
    
    
    투자 판단: {investment_decision}
    
    === 보고서 형식 ===
    
    # 위성 스타트업 투자 평가 보고서
    
    ## Executive Summary
    
    ## 1. 회사 개요
    
    ## 2. 기술력 평가
    ### 2.1 핵심 기술
    ### 2.2 기술적 강점
    ### 2.3 기술 리스크
    ### 2.4 Scorecard 평가
    
    ## 3. 시장성 분석
    ### 3.1 시장 규모 및 성장률
    ### 3.2 시장 기회
    ### 3.3 Scorecard 평가
    
    ## 4. 경쟁 환경
    
    ## 5. 투자 의견
    ### 5.1 최종 판단
    ### 5.2 투자 근거
    ### 5.3 주요 리스크
    
    ## 6. 결론
    """

    result = llm.invoke(report_prompt)
    state["report"] = result.content

    print("보고서 생성 완료")
    return state

## LangGraph 구성

In [ ]:
def create_workflow():
    graph = StateGraph(AgentState)

    graph.add_node("startup_search", startup_search_agent)
    graph.add_node("tech_summary", tech_summary_agent)
    graph.add_node("market_eval", market_eval_agent)
    graph.add_node("investment_decision", investment_decision_agent)
    graph.add_node("report_writer", report_writer_agent)

    graph.set_entry_point("startup_search")

    graph.add_edge("startup_search", "tech_summary")
    graph.add_edge("tech_summary", "market_eval")
    graph.add_edge("market_eval", "investment_decision")

    graph.add_conditional_edges(
        "investment_decision",
        should_continue,
        {"report_writer": "report_writer", "startup_search": "startup_search"},
    )

    graph.set_finish_point("report_writer")

    return graph.compile()